In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.style
import matplotlib as mpl
from common_functions import foo


mpl.style.use('classic')

%load_ext autoreload
%autoreload 2

results = "results"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
from common_functions import latexify, format_axes, tableau20blind
latexify(14, 8)

In [4]:
lastKDays = 30
df = foo(lastKDays, results, True)
df.groupby(["reg"]).mean()

,kout,kin,day,rmse,lastKDays,seed
reg,,,,,,
gp,2.498016,2.001424,163.983420,20.665598,30.0,-1.000000
gp_rd,2.499959,1.999573,163.968984,24.380714,30.0,2.000305
knn,2.500000,2.000000,164.000000,30.679426,30.0,-1.000000
knn_rd,2.500000,2.000000,164.000000,30.541583,30.0,2.000000
qbc,2.500000,2.000000,164.000000,22.671395,30.0,-1.000000
qbc_rd,2.500000,2.000000,164.000000,24.136366,30.0,2.000000


In [49]:
regs = df["reg"].unique()
d = {}
for reg1 in regs:
    for reg2 in regs:
        if reg1 == reg2:
            continue
        td = {}
        for ix, tempdf in df.groupby(["kout", "kin"]):
            tempdf1 = tempdf.groupby("reg").mean()
            reg1_rmse = tempdf1.loc[reg1]["rmse"]
            reg2_rmse = tempdf1.loc[reg2]["rmse"]
            td[ix] = 100*(reg1_rmse - reg2_rmse)/reg1_rmse # want max
        d[(reg1, reg2)] = td
        
"""(reg1, reg2) --> % improvement in rmse of reg2 w.r.t. reg1, i.e. 100*(reg1_rmse - reg2_rmse)/reg1_rmse"""

'(reg1, reg2) --> % improvement in rmse of reg2 w.r.t. reg1, i.e. 100*(reg1_rmse - reg2_rmse)/reg1_rmse'

In [50]:
list(d.items())[:2]

[(('gp', 'qbc'),
  {(0, 0): -20.239991001699266,
   (0, 1): -10.703729228562944,
   (0, 2): -29.301479380210438,
   (0, 3): -32.976837139192114,
   (0, 4): -13.575576176478815,
   (1, 0): 7.352603126617488,
   (1, 1): 20.621209195732355,
   (1, 2): -9.329435188960575,
   (1, 3): -14.749795060336828,
   (1, 4): -17.77212535765178,
   (2, 0): 2.192475568094852,
   (2, 1): 17.707096126129418,
   (2, 2): -15.32110310396753,
   (2, 3): -1.634686620178661,
   (2, 4): -52.2645837455235,
   (3, 0): -26.47718897529197,
   (3, 1): -17.83209653951583,
   (3, 2): -4.246900261573831,
   (3, 3): -20.764957493047476,
   (3, 4): -12.705651073852247,
   (4, 0): -16.511272602292237,
   (4, 1): -0.37351935338433406,
   (4, 2): -31.082046012962643,
   (4, 3): -25.87356182957577,
   (4, 4): -31.756858939106717,
   (5, 0): -3.9533619824122948,
   (5, 1): 24.49819133017378,
   (5, 2): -16.078657604558966,
   (5, 3): -0.6555534163499491,
   (5, 4): -11.161359867290807}),
 (('gp', 'knn'),
  {(0, 0): -64.516218

# Best Performance %

In [52]:
def best_perf(d, reg1, reg2):
    perf = d[(reg1, reg2)]
    return max(perf.items(), key=lambda x:x[1])

In [60]:
best_perf(d, 'gp_rd', 'qbc') #best performance of qbc over gp_rd

((2, 1), 33.79220977536657)

In [59]:
best_perf(d, 'qbc_rd', 'qbc') # best performance of qbc over qbc_rd

((0, 4), 23.885465892737745)

In [58]:
best_perf(d, 'qbc', 'gp') # best performance of gp over qbc, i.e. best gp_rmse (100-34) percent of qbc_rmse

((2, 4), 34.32484590958602)

# Worst Performance %

In [61]:
def worst_perf(d, reg1, reg2):
    perf = d[(reg1, reg2)]
    return min(perf.items(), key=lambda x:x[1])

In [64]:
worst_perf(d, 'gp_rd', 'qbc') # worst performance of qbc over gp_rd

((0, 0), -37.850665251738924)

In [63]:
worst_perf(d, 'qbc_rd', 'qbc') # worst performance of qbc over qbc_rd

((3, 0), -10.5114577521837)

In [65]:
worst_perf(d, 'qbc', 'gp') # worst performance of gp over qbc, i.e. best gp_rmse (100-34) percent of qbc_rmse

((5, 1), -32.4471582360441)